In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder

from scipy.io import arff

In [616]:
file = "datasets/haggis_data.arff"

arff_file = arff.loadarff(file)
data = pd.DataFrame(arff_file[0])
# Decode byte strings to regular strings
data = data.map(lambda x: x.decode() if isinstance(x, bytes) else x)
data

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,hairy,brown,large,hard,1,3.25,1
1,hairy,green,?,hard,1,4.22,1
2,?,red,small,soft,0,1.27,0
3,hairy,green,large,hard,1,3.55,1
4,smooth,red,small,soft,0,2.13,0
5,smooth,green,large,soft,1,2.67,0
6,hairy,?,large,soft,0,3.77,1


Pour plus tard, on charge maintenant les bases de données de test, afin de pouvoir appliquer proprement LabelEncoder (si on le fait séparément, des attributs vont être représenté par des classes différentes)

In [ ]:
file = "datasets/haggis_un_seul_test.arff"
arff_file = arff.loadarff(file)
data_one_test = pd.DataFrame(arff_file[0])
data_one_test = data_one_test.map(lambda x: x.decode() if isinstance(x, bytes) else x)
data_one_test

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,smooth,red,large,hard,1,3.25,1


In [ ]:
file = "datasets/haggis_test.arff"
arff_file = arff.loadarff(file)
data_test = pd.DataFrame(arff_file[0])
data_test = data_test.map(lambda x: x.decode() if isinstance(x, bytes) else x)
data_test

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,smooth,red,large,hard,1,3.25,1
1,hairy,brown,small,hard,1,2.56,0
2,smooth,green,small,hard,1,3.05,1
3,hairy,red,large,soft,0,2.05,1


Scikit-learn ne gère pas les bases de données où il y plusieurs type de données. https://scikit-learn.org/stable/modules/naive_bayes.html

On discrètise alors l'attribut length afin de pouvoir appliquer une classification catégorielle. De plus, il faut que chaque catégorie soit rerpésenté par un entier.
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html

In [619]:
data_all = pd.concat([data, data_one_test, data_test])
# Define the bins and labels
bins = [1, 2, 3, 5]
labels = ['short', 'medium', 'long']

# Discretize the length column
data_all['length'] = pd.cut(data_all['length'], bins=bins, labels=labels)
data_all.replace("?", np.nan, inplace=True)
data_all

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,hairy,brown,large,hard,1,long,1
1,hairy,green,NaN,hard,1,long,1
2,NaN,red,small,soft,0,short,0
3,hairy,green,large,hard,1,long,1
4,smooth,red,small,soft,0,medium,0
5,smooth,green,large,soft,1,medium,0
6,hairy,NaN,large,soft,0,long,1
0,smooth,red,large,hard,1,long,1
0,smooth,red,large,hard,1,long,1
1,hairy,brown,small,hard,1,medium,0


In [620]:
le = LabelEncoder()
data_all = data_all.apply(le.fit_transform)
data_all

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,0,0,0,0,1,0,1
1,0,1,2,0,1,0,1
2,2,2,1,1,0,2,0
3,0,1,0,0,1,0,1
4,1,2,1,1,0,1,0
5,1,1,0,1,1,1,0
6,0,3,0,1,0,0,1
0,1,2,0,0,1,0,1
0,1,2,0,0,1,0,1
1,0,0,1,0,1,1,0


In [621]:
data = data_all.iloc[:data.shape[0], :]
data_one_test = data_all.iloc[data.shape[0]:data.shape[0]+data_one_test.shape[0], :]
data_test = data_all.iloc[data.shape[0]+data_one_test.shape[0]:, :]

data

,skin,colour,size,flesh,eats_shortbread,length,is_haggis
0,0,0,0,0,1,0,1
1,0,1,2,0,1,0,1
2,2,2,1,1,0,2,0
3,0,1,0,0,1,0,1
4,1,2,1,1,0,1,0
5,1,1,0,1,1,1,0
6,0,3,0,1,0,0,1


Attention, le tirage au sort de l'ensemble de test peut donner 3 instances faisant partie de la même classe. Dans ce cas, on peut réexecuter la cellues (les variables data, X et y ne sont pas modifié dans la suite de ce notebook).

In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

_, X_test, _, y_test = train_test_split(X, y, test_size=0.4)

gnb = CategoricalNB()
model = gnb.fit(X, y)
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[1, 0],
       [0, 2]])

Résultats de la classification sur Weka :
```=== Run information ===

Scheme:       weka.classifiers.bayes.NaiveBayes 
Relation:     Haggis
Instances:    7
Attributes:   7
              skin
              colour
              size
              flesh
              eats_shortbread
              length
              is_haggis
Test mode:    split 60.0% train, remainder test

=== Classifier model (full training set) ===

Naive Bayes Classifier

                   Class
Attribute              0      1
                  (0.44) (0.56)
================================
skin
  hairy               1.0    5.0
  smooth              3.0    1.0
  [total]             4.0    6.0

colour
  brown               1.0    2.0
  green               2.0    3.0
  red                 3.0    1.0
  [total]             6.0    6.0

size
  small               3.0    1.0
  large               2.0    4.0
  [total]             5.0    5.0

flesh
  soft                4.0    2.0
  hard                1.0    4.0
  [total]             5.0    6.0

eats_shortbread
  0                   3.0    2.0
  1                   2.0    4.0
  [total]             5.0    6.0

length
  mean             1.9667 3.8104
  std. dev.        0.4014 0.4077
  weight sum            3      4
  precision        0.4917 0.4917



Time taken to build model: 0 seconds

=== Evaluation on test split ===

Time taken to test model on test split: 0 seconds

=== Summary ===

Correctly Classified Instances           2               66.6667 %
Incorrectly Classified Instances         1               33.3333 %
Kappa statistic                          0.4   
Mean absolute error                      0.3473
Root mean squared error                  0.5779
Relative absolute error                 62.5085 %
Root relative squared error            100.0874 %
Total Number of Instances                3     

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,500    0,000    1,000      0,500    0,667      0,500    1,000     1,000     0
                 1,000    0,500    0,500      1,000    0,667      0,500    0,750     0,500     1
Weighted Avg.    0,667    0,167    0,833      0,667    0,667      0,500    0,917     0,833     

=== Confusion Matrix ===

 a b   <-- classified as
 1 1 | a = 0
 0 1 | b = 1
```

***Question 1***

La méthode de classification de Bayes repose sur la probabilité conditionnelle des attributs par rapport à la classe. Lorsqu'un attribut est catégoriel (comme ici "skin" avec les modalités "hairy" et "smooth"), on utilise la fréquence des observations pour estimer les probabilités conditionnelles. Lorsqu'une catégorie est absente dans une des classes (i.e., une probabilité égale à zéro), cela peut poser problème pour la classification, car une probabilité nulle entraîne la nullité de la probabilité jointe. Pour éviter cela, le lissage de Laplace est appliqué. Le lissage de Laplace consiste à ajouter une petite constante (ici 1) à chaque fréquence.
Le tableau donnée par Weka représente donc les fréquences de chaque attribut, en ajoutant 1 à chaque donnée pour éviter qu'il y ai un veto de l'attribut hairy pour la classe 0 et de l'attribut smooth pour la classe 1.
Scikit-learn quant à lui, ajoute un paramètre de lissage pour le calcul de chaque probabilité : https://scikit-learn.org/stable/modules/naive_bayes.html#probability-calculation

***Question 2***

Pour les résultats de Weka, on a 2 éléments correctement prédits, ainsi que 1 faux positif. Même si Weka fait l'apprentissage sur l'ensemble de la base de données, cette dernière ne possède que 7 élements, et elle contient des valeurs manquantes. Cela cause donc des approximations lors de l'apprentissage.
Pour les résultats avec python, le modèle prédit toujours correctement.

***Question 3***

1. Le caractère "?" indique une valeur manquante.
2. La méthode de Bayes se base de probabilités conditionnelles calculées à partir des fréquences. Ainsi les valeurs manquantes sont juste ignorées par Weka comme on l'observe pour l'attribut skin. Des 1 ont été ajoutés à cause du lissage de Laplace, mais si on les retire, on a 4 haggis "hairy" et 2 haggis "smooth", comme dans la base de données.
Dans le programme Python, les valeurs manquantes sont considérées comme une catégorie supplémentaire.
3. Weka suppose généralement que les attributs numériques suivent une distribution normale. Pour le modèle Naïve Bayes, Weka utilise la moyenne et l'écart-type pour estimer la probabilité conditionnelle des valeurs continues. Pour les valeurs catégorielles, elles sont traitées comme des niveaux distincts. Pour chaques catégories, les probabilités conditionnelles sont calculées, avec un lissage de Laplace pour éviter les probabilités nulles.
En python, on discrètise cet attribut pour le rendre catégoriel.

***Question 4***

On classer cette instance : x=(skin=smooth, color=red, size=large, flesh=hard, eats shortbread=yes, length=3.25)
On remarque dans la base de données que tous les animaux sans poil, et tout ceux rouges ne sont pas des haggis. Cependant tous les animaux avec de la chaire dure, et tous ceux qui mesure plus de 2.75 sont des haggis (et la réciproque est vraie). Pour les deux critères restants, la catégorie dont la nouvelle instance fait partie est constituée par une majorité de haggis. Donc on classe x comme un haggis.

Après vérification sur Weka, cette instance est classé comme un haggis.

In [623]:
data_one_test
X_one_test = data_one_test.iloc[:, :-1]
y_one_test = data_one_test.iloc[:, -1]

y_pred = model.predict(X_one_test)
print("Classe de x :", y_one_test.values[0])
print("Classe prédite :", y_pred[0])

Classe de x : 1
Classe prédite : 1


***Question 5***

In [624]:
def densite_de_proba(x, sigma, mu):
    return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2))

p_length_no_haggis = densite_de_proba(3.25, 0.4014, 1.9667)
p_length_haggis = densite_de_proba(3.25, 0.4077, 3.8104)

print("P(length = 3.25 | no haggis) =", p_length_no_haggis)
print("P(length = 3.25 | haggis) =", p_length_haggis)

P(length = 3.25 | no haggis) = 0.005995542897034256
P(length = 3.25 | haggis) = 0.380452056303937


In [625]:
num_p_haggis = 1/6 * 1/6 * 4/5 * 4/6 * 4/6 * p_length_haggis*4/7
num_p_no_haggis = 3/4 * 3/6 * 2/5 * 1/5 * 2/5 * p_length_no_haggis*3/7
p_x = num_p_haggis + num_p_no_haggis

p_haggis = num_p_haggis / p_x
p_no_haggis = num_p_no_haggis / p_x
print("P(haggis | x) =", p_haggis)
print("P(no haggis | x) =", p_no_haggis)

P(haggis | x) = 0.9858429149931005
P(no haggis | x) = 0.014157085006899611


***Question 6***

Weka a classé l'instance x comme un haggis, ce qui correspond aux calculs.
La matrice de confusion obtenue est : 
```
 a b   <-- classified as
 1 0 | a = 0
 1 2 | b = 1
```
On a donc 75% de succès avec un faux négatif.

In [626]:
X_test_final = data_test.iloc[:, :-1]
y_test_final = data_test.iloc[:, -1]

y_pred = model.predict(X_test_final)
confusion_matrix(y_test_final, y_pred)

array([[0, 1],
       [1, 2]])

Python a de moins bon résultats, avec 50% de taux de succès, un faux négatif et un faux positif.